# Module 8: Image Generation and GANs
## Tujuan
- Membuat gambar baru menggunakan Generative Adversarial Networks

In [ ]:
!pip install tensorflow matplotlib
import tensorflow as tf
import matplotlib.pyplot as plt
# Contoh implementasi GAN sederhana akan dibuat di sini